In [31]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

# --- Data setup ---
data = [
    ["Piotr Lesiecki", "00:27:08", "00:05:22", "02:05:53", "00:02:34", "01:20:52"],
    ["Nicolas Mallebay", "00:28:11", "00:04:26", "02:07:43", "00:02:55", "01:18:39"],
    ["Nicolas Lange-Berteaux", "00:30:57", "00:04:45", "02:09:05", "00:02:58", "01:22:20"],
    ["Jesse Massez", "00:31:54", "00:05:43", "02:09:22", "00:03:10", "01:20:20"],
    ["Jonathon Richard Churchill", "00:33:15", "00:05:17", "02:11:17", "00:03:26", "01:18:31"],
    ["Steffen De Geyter", "00:32:47", "00:05:55", "02:06:55", "00:03:49", "01:22:59"],
    ["Álvaro Heredero Perera", "00:32:02", "00:05:47", "02:08:52", "00:02:45", "01:25:44"],
    ["Renaud Lerooy", "00:30:29", "00:05:14", "02:12:36", "00:04:28", "01:22:39"],
    ["Daan Deberdt", "00:25:41", "00:04:52", "02:09:11", "00:05:09", "01:30:50"],
    ["Tom Davey", "00:31:33", "00:05:17", "02:07:26", "00:02:47", "01:28:47"],
]
columns = ["Athlete", "Swim_str", "T1_str", "Bike_str", "T2_str", "Run_str"]
df = pd.DataFrame(data, columns=columns)

def parse_time_str(t):
    parts = t.split(":")
    if len(parts) == 3:
        h, m, s = parts
    else:
        h = 0
        m, s = parts
    return pd.Timedelta(hours=int(h), minutes=int(m), seconds=int(s))

for col in ["Swim_str", "T1_str", "Bike_str", "T2_str", "Run_str"]:
    df[col.replace("_str", "")] = df[col].apply(parse_time_str)

df["After Swim"] = df["Swim"]
df["After T1"] = df["After Swim"] + df["T1"]
df["After Bike"] = df["After T1"] + df["Bike"]
df["After T2"] = df["After Bike"] + df["T2"]
df["After Run"] = df["After T2"] + df["Run"]

steps = ["After Swim", "After T1", "After Bike", "After T2", "After Run"]
labels = ["Swim", "T1", "Bike", "T2", "Run"]
x_labels = ["Natation", "T1", "Vélo", "T2", "Course à pied"]
athlete_name = "Nicolas Lange-Berteaux"

def seconds_to_mmss(s):
    m, s = divmod(s, 60)
    return f"{m:02d}:{s:02d}"

def seconds_to_hhmmss(s):
    h, remainder = divmod(s, 3600)
    m, s = divmod(remainder, 60)
    return f"{h:d}:{m:02d}:{s:02d}" if h > 0 else f"{m:02d}:{s:02d}"

def mmss_to_seconds(s):
    parts = s.strip().split(":")
    try:
        if len(parts) == 2:
            return int(parts[0])*60 + int(parts[1])
        elif len(parts) == 3:
            return int(parts[0])*3600 + int(parts[1])*60 + int(parts[2])
    except:
        return 0
    return 0

output = widgets.Output()

rank_diff_label = widgets.HTML(
    value="", 
    layout=widgets.Layout(margin="10px 0px 10px 0px", font_weight="bold", font_size="18px", text_align="center")
)

def update_rank_diff_label(rank_diff):
    if rank_diff > 0:
        rank_diff_label.value = f'<span style="color:green; font-weight:bold; font-size:20px;">+{rank_diff} &#9650;</span>'
    elif rank_diff < 0:
        rank_diff_label.value = f'<span style="color:red; font-weight:bold; font-size:20px;">{rank_diff} &#9660;</span>'
    else:
        rank_diff_label.value = '<span style="color:gray; font-weight:bold; font-size:20px;">0</span>'

def update_plot(swim, t1, bike, t2, run):
    with output:
        clear_output(wait=True)

        df_copy = df.copy()

        new_times = {
            "Swim": mmss_to_seconds(swim),
            "T1": mmss_to_seconds(t1),
            "Bike": mmss_to_seconds(bike),
            "T2": mmss_to_seconds(t2),
            "Run": mmss_to_seconds(run),
        }

        idx = df_copy.index[df_copy["Athlete"] == athlete_name][0]
        for label in labels:
            df_copy.at[idx, label] = pd.Timedelta(seconds=new_times[label])

        df_copy["After Swim"] = df_copy["Swim"]
        df_copy["After T1"] = df_copy["After Swim"] + df_copy["T1"]
        df_copy["After Bike"] = df_copy["After T1"] + df_copy["Bike"]
        df_copy["After T2"] = df_copy["After Bike"] + df_copy["T2"]
        df_copy["After Run"] = df_copy["After T2"] + df_copy["Run"]

        classement = pd.DataFrame(index=df_copy["Athlete"], columns=steps)
        for step in steps:
            classement[step] = df_copy[step].dt.total_seconds().rank(method="min").astype(int).values

        plt.figure(figsize=(14, 8))
        cmap = plt.colormaps['tab10'].resampled(len(df_copy))

        for i, row in df_copy.iterrows():
            athlete = row["Athlete"]
            y = [classement.loc[athlete, step] for step in steps]
            color = 'red' if athlete == athlete_name else cmap(i)
            plt.plot(x_labels, y, marker='o', linestyle='-', color=color, alpha=0.5, linewidth=2 if athlete == athlete_name else 1)

            for i_step, step in enumerate(steps):
                if athlete == athlete_name:
                    segment_sec = new_times[labels[i_step]]
                else:
                    segment_sec = row[labels[i_step]].total_seconds()
                if labels[i_step] in ["Bike", "Run"]:
                    seg_label = seconds_to_hhmmss(int(segment_sec))
                else:
                    seg_label = seconds_to_mmss(int(segment_sec))
                plt.text(i_step, y[i_step] - 0.3, seg_label, ha='center', va='top', fontsize=8, color=color)

                if i_step != 0:
                    cumul_sec = row[step].total_seconds()
                    if athlete == athlete_name:
                        cumul_sec = sum(new_times[labels[j]] for j in range(i_step+1))
                    if step in ["After Bike", "After Run"]:
                        cumul_label = seconds_to_hhmmss(int(cumul_sec))
                    else:
                        cumul_label = seconds_to_mmss(int(cumul_sec))
                    plt.text(i_step, y[i_step] + 0.3, f"({cumul_label})", ha='center', va='bottom', fontsize=7, color=color, style='italic')

            plt.text(len(x_labels), y[-1], athlete, va='center', ha='left', fontsize=9, color=color)

        plt.title("Évolution du classement durant la course (top 10 athlètes)")
        plt.xlabel("Portion de la course")
        plt.ylabel("Classement")
        plt.xticks(ticks=range(len(x_labels) + 1), labels=x_labels + [""])
        plt.gca().invert_yaxis()
        plt.grid(True)
        plt.tight_layout()
        plt.show()

        return classement.loc[athlete_name, "After Run"]

slider_bounds = {
    "Swim": {"min": 25 * 60, "max": 35 * 60},
    "T1": {"min": 3 * 60, "max": 7 * 60},
    "Bike": {"min": 2 * 3600, "max": int(2.5 * 3600)},
    "T2": {"min": 3 * 60, "max": 7 * 60},
    "Run": {"min": 75 * 60, "max": 90 * 60},
}

widgets_sliders = {}
widgets_diff_labels = {}

original_row = df[df["Athlete"] == athlete_name].iloc[0]

def format_readout(value):
    return seconds_to_hhmmss(value) if value >= 3600 else seconds_to_mmss(value)

def create_widgets(label):
    val_sec = int(original_row[label].total_seconds())

    slider_widget = widgets.IntSlider(
        min=int(slider_bounds[label]["min"]),
        max=int(slider_bounds[label]["max"]),
        value=val_sec,
        description=f"{label}",
        readout=False,
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='70%', margin='0 10px 0 0')
    )
    
    diff_label = widgets.Label(value="", layout=widgets.Layout(width='70px', margin='0 10px 0 0'))
    time_readout_label = widgets.Label(value=format_readout(val_sec), layout=widgets.Layout(width='80px'))

    def update_diff_label(new_val):
        diff_sec = new_val - val_sec
        sign = "+" if diff_sec > 0 else ""
        if diff_sec == 0:
            diff_label.value = ""
        elif abs(diff_sec) < 60:
            diff_label.value = f"{sign}{diff_sec}s"
        else:
            m, s = divmod(abs(diff_sec), 60)
            diff_label.value = f"{sign}{'-' if diff_sec < 0 else ''}{m}m{s:02d}s"

    def on_slider_change(change):
        if change['name'] == 'value':
            new_val = change['new']
            time_readout_label.value = format_readout(new_val)
            update_diff_label(new_val)
            trigger_update()

    slider_widget.observe(on_slider_change)

    widgets_sliders[label] = slider_widget
    widgets_diff_labels[label] = diff_label

    box = widgets.HBox([slider_widget, diff_label, time_readout_label])
    return box

def trigger_update():
    vals = {label: format_readout(widgets_sliders[label].value) for label in labels}
    final_rank = update_plot(vals["Swim"], vals["T1"], vals["Bike"], vals["T2"], vals["Run"])

    classement_original = df.copy()
    classement_original["Rank"] = classement_original["After Run"].dt.total_seconds().rank(method="min").astype(int)
    orig_rank = classement_original.loc[classement_original["Athlete"] == athlete_name, "Rank"].values[0]

    rank_diff = orig_rank - final_rank
    update_rank_diff_label(rank_diff)

def reset_sliders(_):
    for label in labels:
        val_sec = int(original_row[label].total_seconds())
        widgets_sliders[label].value = val_sec

reset_button = widgets.Button(description="Reset", button_style='warning', layout=widgets.Layout(width='100px', margin='0 0 10px 0'))
reset_button.on_click(reset_sliders)

slider_boxes = [create_widgets(label) for label in labels]

ui = widgets.VBox([reset_button, rank_diff_label] + slider_boxes + [output])

trigger_update()

display(ui)
